In [1]:
# RECULSIVE COPY OF FILES FROM ONE DIRECTORY TO ANOTHER

import os
import shutil
import random

# Define the source directory
src_dir = r'C:\IMPORTANT\HP_PC\CODING\HUGGINGFACE\modelhub managing\mixed-SDXL-Random-Outputs'

# Define the destination directory
dst_dir = './combined_all_images'
groups_dir = './combined_groups'
final_vids_dir = './final_vids'

# Create the destination directories if they don't exist if it doesn't exist
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)    
if not os.path.exists(groups_dir):
    os.makedirs(groups_dir)
if not os.path.exists(final_vids_dir):
    os.makedirs(final_vids_dir)    

    


# Loop through all files in the source directory and its subdirectories
for root, dirs, files in os.walk(src_dir):
    for file in files:
        # Check if the file is a .png image
        if file.endswith('.png'):
            # Construct the full path of the source file
            src_file = os.path.join(root, file)
            # Construct the full path of the destination file
            dst_file = os.path.join(dst_dir, file)
            # Copy the file to the destination directory
            shutil.copy(src_file, dst_file)

# Count the number of .png files in the destination directory
imgFilesNo = len([name for name in os.listdir(dst_dir) if name.endswith('.png')])   
print(f'There are {imgFilesNo} .png files copied to the destination directory.') 


# Get a list of all .png files in the destination directory
png_files = [f for f in os.listdir(dst_dir) if f.endswith('.png')]

# Shuffle the list of .png files
random.shuffle(png_files)

# Split the list of .png files into groups of 10
png_groups = [png_files[i:i+10] for i in range(0, len(png_files), 10)]

# Loop through each group of .png files
for i, group in enumerate(png_groups):
    # Create a new directory for the group
    group_dir = os.path.join(groups_dir, f'group_{i+1}')
    os.makedirs(group_dir)
    # Copy each .png file in the group to the new directory
    for file in group:
        src_file = os.path.join(dst_dir, file)
        dst_file = os.path.join(group_dir, file)
        shutil.copy(src_file, dst_file)

There are 212 .png files copied to the destination directory.


In [2]:

from moviepy.editor import VideoFileClip, concatenate_videoclips
from moviepy.video.fx.fadein import fadein
from moviepy.video.fx.fadeout import fadeout
# from moviepy.video.compositing.transitions import crossfadein, crossfadeout, slide_in
from pathlib import Path

import shutil
import os
import random

homePath = Path(r"C:\Users\Administrator\Pictures\vidgen_slideshow")
os.chdir(homePath)

# specify the directory path
dir_path = homePath / 'combined_groups'
final_Path = homePath / 'final_vids'

# get a list of all files and directories in the specified directory
files_and_dirs = os.listdir(dir_path)

# filter out only the directories
image_groups_dirs_list = [d for d in files_and_dirs if os.path.isdir(os.path.join(dir_path, d))]
print('There are ' + str(len(image_groups_dirs_list)) + ' image groups')

# Create an empty list to store completions
completedDirList = []

# Loop through each directory and generate a video for each group
for group in image_groups_dirs_list:
    # Define the directory containing PNG images
    source_Imgs_dir = dir_path / group

    # Change the working directory to the source directory
    os.chdir(homePath)
    os.chdir(source_Imgs_dir)

    print(f"Starting to create Static Videos for {group}")
    
    # Create an output directory for the final videos if it does not exist
    !pwsh -c "if (!(Test-Path -Path final_videos)) { $null = New-Item -ItemType Directory -Path final_videos }"
    
    finalVideosDir = source_Imgs_dir / 'final_videos'
    
    # Generate a video from all PNG images in the source directory
    !pwsh -c "Get-ChildItem -Filter *.png | ForEach-Object -Parallel { ffmpeg -loglevel quiet -loop 1 -i ($_.FullName) -c:v libx264 -t 6 -r 30 -pix_fmt yuv420p ('final_videos\' + $_.BaseName + '.mp4')}"
    
    print("Finished Creating Static Videos from images")
    
    # Excute a command for Zooming & panning to top center
    # change the current working directory to one level up
    os.chdir(homePath)
    os.chdir(finalVideosDir)
    !pwsh -c "Get-ChildItem -Filter *.mp4 | ForEach-Object -Parallel  { ffmpeg -loglevel quiet -i ($_.FullName) -vf scale=8240x10240,zoompan=z=pzoom+0.003:x='iw/2-iw/zoom/2':y=0:d=1:s=824x1024:fps=30 ($_.BaseName + '_pan.mp4')}"
    
    
    # Load video clips ending with "_pan.mp4"
    
    video_clips = []
    os.chdir(homePath)
        
    # List the video files in the directory
    video_files = [filename for filename in os.listdir(finalVideosDir) if filename.endswith("_pan.mp4")]
    
    # Choose a random video to skip the fadein effect
    video_to_skip_fadein = random.choice(video_files)
    
    # make a clip for the video to skip the fadein effect
    firstClip = VideoFileClip(os.path.join(finalVideosDir, video_to_skip_fadein))
    # Apply faedeout effect to the clip
    firstClip = fadeout(clip=firstClip, duration=0.25, final_color=(0, 0, 0))
    
    for filename in video_files:
        if filename != video_to_skip_fadein:
           # Make a clip for the video
           clip = VideoFileClip(os.path.join(finalVideosDir, filename))
           # Apply the fadeout effect to each clip
           clip = fadeout(clip=clip, duration=0.25, final_color=(0, 0, 0))    
           # Apply the fadein effect to each clip 
           clip = fadein(clip=clip, duration=0.25, initial_color=(0, 0, 0))
           
           video_clips.append(clip)
    
    # Shuffle the video clips
    random.shuffle(video_clips)
    
    # Add the first clip at the beginning of the list
    video_clips.insert(0, firstClip)
                 
    # Concatenate the video clips into a single video
    final_clip = concatenate_videoclips(video_clips)
    
    
    # Save the final video to a file
    os.chdir(source_Imgs_dir)
    finalVideoName = 'final_video_' + group + '.mp4'
    final_clip.write_videofile(finalVideoName)
    
    os.chdir(homePath)
    finalVidPathTemp = source_Imgs_dir / finalVideoName
    # Copy the final video to the finalVideosDir directory
    shutil.copy(finalVidPathTemp, final_Path)
    
    # Delete the finalVideosDir directory
    shutil.rmtree(finalVideosDir)

    # Add the group to the completedDirList
    completedDirList.append(group)
    
    print(f"Finished Directories: {len(completedDirList)} \nRemaining Directories: {len(image_groups_dirs_list) - len(completedDirList)} \nAll Directories: {len(image_groups_dirs_list)}")

There are 22 image groups
Starting to create Static Videos for group_1
Finished Creating Static Videos from images
Moviepy - Building video final_video_group_1.mp4.
Moviepy - Writing video final_video_group_1.mp4



Moviepy - Done !
Moviepy - video ready final_video_group_1.mp4
Finished Directories: 1 
Remaining Directories: 21 
All Directories: 22
Starting to create Static Videos for group_10
Finished Creating Static Videos from images
Moviepy - Building video final_video_group_10.mp4.
Moviepy - Writing video final_video_group_10.mp4



Moviepy - Done !
Moviepy - video ready final_video_group_10.mp4
Finished Directories: 2 
Remaining Directories: 20 
All Directories: 22
Starting to create Static Videos for group_11


In [7]:
# TO VIDEO ALL CODE

from moviepy.editor import VideoFileClip, concatenate_videoclips
from moviepy.video.fx.fadein import fadein
from moviepy.video.fx.fadeout import fadeout
# from moviepy.video.compositing.transitions import crossfadein, crossfadeout, slide_in




import os
import random

# Define the directory containing PNG images
source_Imgs_dir = r"C:\Users\Administrator\Pictures\vidgen_slideshow\combined_groups\group_1"


# Define a function to get the dimensions of a video
def get_video_dimensions(video_path):
    video = VideoFileClip(video_path)
    width = video.size[0]
    height = video.size[1]
    return [width, height]


# Change the working directory to the source directory
os.chdir(source_Imgs_dir)

# Create an output directory for the final videos if it does not exist
# !pwsh -c "if (!(Test-Path -Path final_videos)) { New-Item -ItemType Directory -Path final_videos }"

finalVideosDir = os.path.join(source_Imgs_dir, "final_videos")

# Generate a video from all PNG images in the source directory
# !pwsh -c "Get-ChildItem -Filter *.png | ForEach-Object -Parallel { ffmpeg -loglevel quiet -loop 1 -i ($_.FullName) -c:v libx264 -t 6 -r 30 -pix_fmt yuv420p ('final_videos\' + $_.BaseName + '.mp4')}"

print("Finished Creating Static Videos from images")

# Excute a command for Zooming & panning to top center
os.chdir(finalVideosDir)
# !pwsh -c "Get-ChildItem -Filter *.mp4 | ForEach-Object -Parallel  { ffmpeg -loglevel quiet -i ($_.FullName) -vf scale=8240x10240,zoompan=z=pzoom+0.003:x='iw/2-iw/zoom/2':y=0:d=1:s=824x1024:fps=30 ($_.BaseName + '_pan.mp4')}"


# Load video clips ending with "_pan.mp4"

video_clips = []

# List the video files in the directory
video_files = [filename for filename in os.listdir(finalVideosDir) if filename.endswith("_pan.mp4")]

# Choose a random video to skip the fadein effect
video_to_skip_fadein = random.choice(video_files)

# make a clip for the video to skip the fadein effect
firstClip = VideoFileClip(os.path.join(finalVideosDir, video_to_skip_fadein))
# Apply faedeout effect to the clip
firstClip = fadeout(clip=firstClip, duration=0.25, final_color=(0, 0, 0))

for filename in video_files:
    if filename != video_to_skip_fadein:
       # Make a clip for the video
       clip = VideoFileClip(os.path.join(finalVideosDir, filename))
       # Apply the fadeout effect to each clip
       clip = fadeout(clip=clip, duration=0.25, final_color=(0, 0, 0))    
       # Apply the fadein effect to each clip 
       clip = fadein(clip=clip, duration=0.25, initial_color=(0, 0, 0))
       
       video_clips.append(clip)

# Shuffle the video clips
random.shuffle(video_clips)

# Add the first clip at the beginning of the list
video_clips.insert(0, firstClip)
             
# Concatenate the video clips into a single video
final_clip = concatenate_videoclips(video_clips)


# Save the final video to a file
os.chdir(source_Imgs_dir)
final_clip.write_videofile("final_video_v4.mp4")

# Delete the finalVideosDir directory
# !pwsh -c "Remove-Item -Recurse final_videos"

Finished Creating Static Videos from images
Moviepy - Building video final_video_v4.mp4.
Moviepy - Writing video final_video_v4.mp4



Moviepy - Done !
Moviepy - video ready final_video_v4.mp4


In [44]:
from moviepy.editor import VideoFileClip

def get_video_dimensions(video_path):
    video = VideoFileClip(video_path)
    width = video.size[0]
    height = video.size[1]
    return [width, height]

vidPath = r'C:\Users\Administrator\Documents\temporally\SUNBANK MABATI\products3D\corrugated\test\canvas_resized_dir\few\portraits\mynes (1).mp4'

[vidWidth, vidHeight] = get_video_dimensions(vidPath)
print(vidWidth*2)

1648


In [3]:
name = "Alice"
age = 25
print("My name is %s and I'm %d years old." % (name, age))
!pwsh -c "Get-ChildItem -Filter *.mp4 | ForEach-Object -Parallel  {$xPosition = Get-Random -Minimum 8240/2 -Maximum 8240; $yPosition = Get-Random -Minimum 10240/2 -Maximum 10240; ffmpeg -loglevel quiet -i ($_.FullName) -vf scale=8240x10240,zoompan=z=pzoom+0.003:x=$xPosition:y=$yPosition:d=1:s=824x1024:fps=30 ($_.BaseName + '_pan.mp4')}"

My name is Alice and I'm 25 years old.
